# Run PncA WandB Sweep

In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd

import torch
from torch_geometric.data import Data

import wandb

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running via SSH'

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


True

In [3]:
full_dataset = torch.load('full_orig_dataset.pth')

### Set Up Params and Sweep Config

In [4]:
seed = 42
np.random.seed(seed)
random.seed(seed)

# logging params (only used for wandb metrics)
n_samples = 10000
# cutoff_distance = 6.3  

# gcn params
num_node_features = 12
batch_size = 256
# hidden_channels = 64
# learning_rate = 0.001
# wd = 5e-5
epochs = 3000

wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [5]:
# First sweep:

sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'Test Accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'hidden_channels': {
        'values': [32, 64, 128, 256]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 1e-8,
        'max': 1e-2
        },
    'dropout': {
          'values': [0.2, 0.4, 0.5, 0.6, 0.8]
        },
    'cutoff_distance': {
        'distribution': 'uniform',
        'min': 3.5,
        'max': 10 
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-8,
        'max': 1e-1
      },
    }

sweep_config['parameters'] = parameters_dict

# Second / third sweep:

parameters_dict.update({
    'hidden_channels': {
        'values': [64, 128, 192, 256, 320, 384]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 5e-7, 
        'max': 1e-2
        },
    'dropout': {
          'values': [0.4, 0.5, 0.6, 0.8]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-2 
        },
    })

# Fourth / Fifth sweep:

parameters_dict.update({
    'hidden_channels': {
        'values': [128, 192, 256, 320]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-3
        },
    'dropout': {
          'values': [0.4, 0.5, 0.6, 0.8]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 5e-5,
        'max': 5e-2
        },
    })

# Sixth sweep:

parameters_dict.update({
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-6,
        'max': 5e-3 
        }
    })

# Seventh sweep:

parameters_dict.update({
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 5e-6,
        'max': 5e-4 
        }
    })

# Eight sweep w cutoff varying:

parameters_dict.update({
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 5e-6,
        'max': 1e-3 
        }
    })



In [6]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Test Accuracy'},
 'parameters': {'cutoff_distance': {'distribution': 'uniform',
                                    'max': 10,
                                    'min': 3.5},
                'dropout': {'values': [0.4, 0.5, 0.6, 0.8]},
                'hidden_channels': {'values': [128, 192, 256, 320]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.001,
                                  'min': 5e-06},
                'weight_decay': {'distribution': 'log_uniform_values',
                                 'max': 0.001,
                                 'min': 1e-05}}}


### Define Training Loop

In [7]:
project = "pnca-sweep-1"

In [8]:
# sweep_id = wandb.sweep(sweep_config, project=project)
# sweep_id = '18ili8gf'
# sweep_id = 'z95xpbwd'
# sweep_id = 'tr8mwg42'
# sweep_id = '2hsfpqs6'
# sweep_id = 'gt2h3xwl'
# sweep_id = 'y0a18l3k'
# sweep_id = 'rl3tm5la'
sweep_id = 'f2b4l8u4'


In [9]:
def sweep_run():

    with wandb.init() as run:
        config = run.config

        model = run_model.pnca_simpleGCN(
            # sequences=sequences_dict,
            sequences=None,
            self_loops = False,
            cutoff_distance = config.cutoff_distance,
            edge_weight_func = '1-(dist/cutoff)',
            # edge_weight_func = 'none',
            batch_size = batch_size,
            num_node_features = num_node_features,
            hidden_channels = config.hidden_channels,
            learning_rate = config.learning_rate,
            wd = config.weight_decay,
            dropout = config.dropout,
            lr_scheduling=False,
            epochs = epochs,
            dataset = full_dataset,
            normalise_ews=True,
            wandb_params={
                'use_wandb': False,
                'sweep': True
            }
        )

        os.makedirs(f'saved_models/carter_ds_aug/{project}/{sweep_id}', exist_ok=True)
        
        torch.save(model, f'saved_models/carter_ds_aug/{project}/{sweep_id}/{run.name}')

In [10]:
wandb.agent(sweep_id, sweep_run, project = project, count=80)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Agent Starting Run: r73o0smh with config:
wandb: 	cutoff_distance: 8.342651517566143
wandb: 	dropout: 0.8
wandb: 	hidden_channels: 128
wandb: 	learning_rate: 3.190713200531528e-05
wandb: 	weight_decay: 2.144764626793304e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: dy

Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.5264, Test Acc: 0.5103, Train Loss: 0.6870, Test Loss: 0.6875
Epoch: 020, Train Acc: 0.5189, Test Acc: 0.4923, Train Loss: 0.6804, Test Loss: 0.6797
Epoch: 030, Train Acc: 0.5199, Test Acc: 0.4733, Train Loss: 0.6722, Test Loss: 0.6696
Epoch: 040, Train Acc: 0.6847, Test Acc: 0.6707, Train Loss: 0.6625, Test Loss: 0.6572
Epoch: 050, Train Acc: 0.7136, Test Acc: 0.6797, Train Loss: 0.6454, Test Loss: 0.6402
Epoch: 060, Train Acc: 0.7116, Test Acc: 0.6837, Train Loss: 0.6271, Test Loss: 0.6213
Epoch: 070, Train Acc: 0.7109, Test Acc: 0.6847, Train Loss: 0.6067, Test Loss: 0.6001
Epoch: 080, Train Acc: 0.7534, Test Acc: 0.7217, Train Loss: 0.5756, Test Loss: 0.5804
Epoch: 090, Train Acc: 0.7621, Test Acc: 0.7240, Train Loss: 0.5504, Test Loss: 0.5586
Epoch: 100, Train Acc: 0.7483, Test Acc: 0.7253, Train Loss: 0.5296, Test Loss: 0.5405
Epoch: 110, Train Acc: 0.7226, Test Acc: 0.7183, Train Loss: 0.5361

Test Accuracy,▂▂▂▁▁▁▁▅▆▆▂▆▆▇▇▆▇▇▇▇▇▇▇▇▇▂▇█▇▇████████▇█
Test F1,▄▄▄▃▂▁▃▂▄▅▂▃▂▅▅▂▆▅▆▃▇▆▄▇▅▄▅▆█▄▇▇▇▆██▇█▇▆
Test Loss,████▇▇▇▇▇▆▆▅▅▅▄▄▄▃▃▃▄▂▃▂▂▆▂▁▄▃▂▂▁▂▄▃▁▂▆▃
Test Sensitivity,███▇▆▄▇▁▃▄▅▂▂▄▄▁▅▃▃▂▆▄▂▅▃▇▃▃▆▂▅▅▄▃▆▆▄▅▇▃
Test Specificity,▁▁▁▁▂▂▁▇▆▆▂▇▇▇▇█▆▇▇█▅▇█▆█▂██▆█▇▆██▆▆▇▆▅█
Train Accuracy,▁▁▁▁▁▁▁▅▆▆▂▅▅▆▆▅▇▇▇▆▆▇▆▇▇▂▆▇▇▆██▇▇▇███▇▇
Train F1,▃▃▃▃▂▁▃▁▄▅▂▃▂▅▅▁▆▅▅▃▇▆▃▇▅▄▄▆▇▃██▆▆██▇█▇▅
Train Loss,██████▇▇▇▇▇▆▆▆▅▆▅▄▄▅▅▃▄▃▃▅▃▂▃▄▂▂▂▂▃▂▁▁▄▂
Train Sensitivity,███▇▆▅▇▁▄▅▆▂▂▄▄▁▅▄▄▂▇▅▂▆▃█▃▄▇▂▆▆▄▄▇▆▅▆▇▃
Train Specificity,▁▁▁▁▂▃▁█▇▆▂▇█▇▇█▇▇▇█▅▇█▇█▂██▆█▇▇██▆▇▇▇▅█
Test Accuracy,0.786


wandb: Agent Starting Run: xu0e5mai with config:
wandb: 	cutoff_distance: 6.17645982460267
wandb: 	dropout: 0.8
wandb: 	hidden_channels: 128
wandb: 	learning_rate: 1.5128666548448246e-05
wandb: 	weight_decay: 4.624525582178961e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6913, Test Loss: 0.6927
Epoch: 020, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6908, Test Loss: 0.6922
Epoch: 030, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6906, Test Loss: 0.6918
Epoch: 040, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6901, Test Loss: 0.6917
Epoch: 050, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6897, Test Loss: 0.6912
Epoch: 060, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6888, Test Loss: 0.6908
Epoch: 070, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6884, Test Loss: 0.6901
Epoch: 080, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6880, Test Loss: 0.6898
Epoch: 090, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6872, Test Loss: 0.6888
Epoch: 100, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6865, Test Loss: 0.6883
Epoch: 110, Train Acc: 0.5266, Test Acc: 0.5110, Train Loss: 0.6857

...

Test Accuracy,▂▂▂▂▂▂▂▂▂▂▁▆▆▆▇▇▇▇▇▇▇▇▇██▇██████████████
Test F1,▄▄▄▄▄▄▄▄▄▃▁▅▄▅▅▅▄▅▆▅▆▅▅▆▆▆▇▇▇▇▇█▇▇▇█████
Test Loss,█████████▇▇▇▇▆▆▅▅▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
Test Sensitivity,██████████▅▃▂▂▂▂▁▁▂▁▂▁▁▂▂▁▂▂▂▂▂▃▂▂▂▃▃▂▂▃
Test Specificity,▁▁▁▁▁▁▁▁▁▁▂▆▇▇▇▇██▇████████████▇███▇▇██▇
Train Accuracy,▁▁▁▁▁▁▁▁▁▁▁▆▆▆▆▆▆▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇█████
Train F1,▂▂▂▂▂▂▂▂▂▁▁▄▃▃▃▄▃▃▅▄▄▄▄▅▅▅▆▅▅▅▆▇▆▆▇▇▇▇▇█
Train Loss,██████████▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁
Train Sensitivity,█████████▇▆▃▂▂▂▂▁▁▃▂▂▂▂▂▂▂▃▃▂▃▃▄▃▃▃▄▄▃▃▄
Train Specificity,▁▁▁▁▁▁▁▁▁▁▂▇▇▇▇▇▇▇▇▇▇███████████████████
Test Accuracy,0.77133


wandb: Agent Starting Run: b4a8b5ux with config:
wandb: 	cutoff_distance: 6.56821464533794
wandb: 	dropout: 0.5
wandb: 	hidden_channels: 320
wandb: 	learning_rate: 3.2748703196807016e-05
wandb: 	weight_decay: 1.5375783935815962e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.


wandb: Ctrl + C detected. Stopping sweep.


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
